In [ ]:
source = "Recommendations";

In [ ]:
using NBInclude
@nbinclude("../ProductionAlphas/Alpha.ipynb");

In [ ]:
ENV["COLUMNS"] = 999999;
ENV["LINES"] = 100;
recommend_related_series = false;

In [ ]:
anime = DataFrame(CSV.File("../../data/raw_data/anime.csv", ntasks = 1))
anime_to_uid = DataFrame(CSV.File("../../data/processed_data/anime_to_uid.csv"))
anime_to_uid = innerjoin(anime_to_uid, anime, on = "anime_id");

In [ ]:
predictions = read_recommendee_alpha("CombineSignals").rating
rating_df = DataFrame(
    "uid" => 0:length(predictions)-1,
    "rating" => predictions,
    "alpha" => predictions - read_recommendee_alpha("UserItemBiases").rating,
    "std" => read_recommendee_alpha("PredictedErrors").rating,
);
rating_df.score = rating_df.rating - 2 * rating_df.std;

In [ ]:
# evaluate our insample predictions
df = get_recommendee_list()
@debug rmse(df.rating, predictions[df.item])
@debug mae(df.rating, predictions[df.item])
@debug r2(df.rating, predictions[df.item])

In [ ]:
# don't recommend shows that the user has already seen before
function dont_recommend!(df, mask)
    df.rating[mask] .= 0
    df.alpha[mask] .= 0
    df.score[mask] .= -Inf
end

dont_recommend!(rating_df, get_recommendee_list().item)
dont_recommend!(rating_df, get_implicit_list().item)

if !recommend_related_series
    related_series =
        get_alpha("ItemCFRelated.strict_relations", "recommendee_inference").rating .!= 0
    dont_recommend!(rating_df, related_series)
end;

In [ ]:
rec_df = innerjoin(anime_to_uid, rating_df, on = "uid");
rec_df = rec_df[
    :,
    ["anime_id", "uid", "title", "genres", "medium", "rating", "alpha", "std", "score"],
];

In [ ]:
length(get_recommendee_list().item) + length(get_implicit_list().item)

In [ ]:
# the model predicts ratings conditional on the user watching the series
# we want to predict ratings for items the user has not already decided to watch
# to mitigate the sampling bias, we first filter to series that the user is likely to be interested in
# TODO find how to send this automatically
min_alpha = 0.2

In [ ]:
sort!(
    filter(x -> x.alpha .> min_alpha, filter(x -> x.medium == "tv", rec_df)),
    :score,
    rev = true,
)

In [ ]:
sort!(
    filter(x -> x.alpha .> min_alpha, filter(x -> x.medium == "tv", rec_df)),
    :score,
    rev = true,
).title[1:30]

In [ ]:
sort!(filter(x -> x.alpha .> 0, filter(x -> x.medium == "tv", rec_df)), :score, rev = true)

In [ ]:
sort!(filter(x -> x.alpha .> 0, filter(x -> x.medium == "tv", rec_df)), :rating, rev = true)

In [ ]:
sort!(filter(x -> x.alpha > min_alpha, rec_df), :score, rev = true)